# Performance Estimates for the Israeli Inventors Dataset

This notebook showcases the use of our precision and recall performance estimators in application to the Israeli inventors benchmark dataset.

Note that the Israeli dataset only covers patents granted between 1963 and 1999. As such, we can only estimate the performance of the current disambiguation algorithm for this time period.

Furthermore, we consider the the Israeli benchmark to be following the "single block" sampling process.

In [1]:
from pv_evaluation.estimators import pairwise_precision_estimator, pairwise_recall_estimator
from pv_evaluation.benchmark import load_israeli_inventors_benchmark

In [ ]:
# `current_disambiguation` should be a pandas series indexed by inventor mention ID (of the form US-<patent_number>-<inventor-sequence_number>)
# and with values corresponding to disambiguated inventor ID.
# Example (not time constrained):
#    rawinventor = pd.read_csv("rawinventor.tsv", sep="\t")
#    rawinventor["mention-id"] = "US" + rawinventor.patent_id.astype(str) + "-" + rawinventor.sequence.astype(str)
#    current_disambiguation = rawinventor.set_index("mention-id")["inventor_id"]

current_disambiguation = None #TODO

In [ ]:
pairwise_precision_estimator(current_disambiguation, load_israeli_inventors_benchmark(), sampling_type="single_block", weights="uniform")

In [ ]:
pairwise_recall_estimator(current_disambiguation, load_israeli_inventors_benchmark(), sampling_type="single_block", weights="uniform")